# YOLOv8_instance segmentation_Mobius dataset

## To do

- Mask to YOLO annotations

- Separate right/left eyes

```
YOLOv8/
├── images/
│   ├── train/
│   └── val/
├── labels/
│   ├── train/
│   └── val/
└── YOLOv8_seg.yaml
```

### Masks to YOLOv8 annotations

```
階層図
output_directory/
│
├── 1/
│   ├── annotation1.txt
│   ├── annotation2.txt
│   └── ...
│
├── 2/
│   ├── annotation1.txt
│   ├── annotation2.txt
│   └── ...
│
├── 3/
│   ├── annotation1.txt
│   ├── annotation2.txt
│   └── ...
│
└── ...
```

In [45]:
import os
import cv2
import numpy as np
from skimage import measure
from tqdm import tqdm

def create_mask(image, color):
    return np.all(image == color, axis=-1)

def apply_mask(image, mask):
    return np.where(mask, 255, 0)

def extract_contours(img, num_points):
    contours = measure.find_contours(img, 0.5)
    points_list = []

    for contour in contours:
        contour_length = len(contour)

        if contour_length > num_points:
            step = contour_length // num_points
            selected_points = [contour[i * step] for i in range(num_points)]
        else:
            selected_points = contour

        points = [[float(point[1]), float(point[0])] for point in selected_points]
        points_list.append(points)

    return points_list

def normalize_coordinates(points_list, img_width, img_height):
    normalized_points_list = []
    for points in points_list:
        normalized_points = [[x / img_width, y / img_height] for x, y in points]
        normalized_points_list.append(normalized_points)
    return normalized_points_list

def write_yolov8_annotation(points_list, class_index, img_width, img_height, file):
    for points in points_list:
        normalized_points = normalize_coordinates([points], img_width, img_height)[0]
        line = f"{class_index} " + " ".join(f"{x} {y}" for x, y in normalized_points) + "\n"
        file.write(line)

def create_yolov8_text(mask_file, yolov8_annotation_file, num_points):
    img = cv2.imread(mask_file, cv2.IMREAD_COLOR)
    img_height, img_width = img.shape[:2]

    mask_blue = create_mask(img, [255, 0, 0])
    mask_green = create_mask(img, [0, 255, 0])
    mask_red = create_mask(img, [0, 0, 255])

    mask_blue_green = np.logical_or(mask_blue, mask_green)
    mask_all_colors = np.logical_or(mask_blue_green, mask_red)

    img_blue = apply_mask(img, mask_blue)
    img_blue_green = apply_mask(img, mask_blue_green)
    img_all_colors = apply_mask(img, mask_all_colors)

    points_list_blue = extract_contours(img_blue, num_points)
    points_list_blue_green = extract_contours(img_blue_green, num_points)
    points_list_all_colors = extract_contours(img_all_colors, num_points)

    with open(yolov8_annotation_file, 'w') as file:
        # クラスインデックス 0: 瞼
        write_yolov8_annotation(points_list_blue, 0, img_width, img_height, file)
        # クラスインデックス 1: 虹彩
        write_yolov8_annotation(points_list_blue_green, 1, img_width, img_height, file)
        # クラスインデックス 2: 瞳
        write_yolov8_annotation(points_list_all_colors, 2, img_width, img_height, file)

def process_mask_files_in_subdir(subdir, output_subdir, num_points, max_files_remaining):
    mask_files = [f for f in os.listdir(subdir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # max_files_remainingに基づいて処理するファイルの数を調整
    if max_files_remaining is not None:
        mask_files = mask_files[:max_files_remaining]

    if not os.path.exists(output_subdir):
        os.makedirs(output_subdir)

    processed_files = 0
    with tqdm(total=len(mask_files), desc=f"Processing {os.path.basename(subdir)}") as pbar:
        for mask_file in mask_files:
            mask_file_path = os.path.join(subdir, mask_file)
            annotation_file_name = os.path.splitext(mask_file)[0] + '.txt'
            yolov8_annotation_file = os.path.join(output_subdir, annotation_file_name)
            
            create_yolov8_text(mask_file_path, yolov8_annotation_file, num_points)
            pbar.update(1)
            processed_files += 1

    return processed_files

def process_all_mask_files(mask_dir, output_dir, num_points, max_files=None):
    subdirs = [d for d in os.listdir(mask_dir) if os.path.isdir(os.path.join(mask_dir, d))]
    total_subdirs = len(subdirs)
    total_processed_files = 0

    print("start")
    for i, subdir_name in enumerate(subdirs, start=1):
        subdir_path = os.path.join(mask_dir, subdir_name)
        output_subdir = os.path.join(output_dir, subdir_name)
        max_files_remaining = max_files - total_processed_files if max_files is not None else None
        
        if max_files_remaining is not None and max_files_remaining <= 0:
            break
        
        print(f"Processing subdir {i} of {total_subdirs}: {subdir_name}")
        processed_files = process_mask_files_in_subdir(subdir_path, output_subdir, num_points, max_files_remaining)
        total_processed_files += processed_files


# マスクファイルを格納したフォルダのパス
mask_dir = r"C:\Users\CorneAI\Desktop\MOBIUS\Masks"

# アノテーションファイルを保存するフォルダのパス
output_dir = r"C:\Users\CorneAI\Desktop\Mobius_for_YOLOv8\labels"

# ポイントの数
num_points = 24

# プロセスの実行
process_all_mask_files(mask_dir, output_dir, num_points, max_files=None)

start
Processing subdir 1 of 35: 1


Processing 1:   0%|          | 0/144 [00:00<?, ?it/s]

Processing 1:  14%|█▍        | 20/144 [00:09<00:59,  2.07it/s]


KeyboardInterrupt: 

In [42]:
import os
import shutil
import numpy as np
from sklearn.model_selection import GroupKFold

def group_k_fold_split(labels_dir, n_splits=10, n_val_splits=3):
    # 各サブディレクトリのパスを取得
    subdirs = [d for d in os.listdir(labels_dir) if os.path.isdir(os.path.join(labels_dir, d))]

    file_paths = []  # ファイルのフルパス
    groups = []  # 各ファイルに対応するグループ（サブディレクトリ名）

    for subdir in subdirs:
        subdir_path = os.path.join(labels_dir, subdir)
        files = [os.path.join(subdir_path, f) for f in os.listdir(subdir_path) if f.endswith('.txt')]
        file_paths.extend(files)
        groups.extend([subdir] * len(files))

    # GroupKFoldを使用して分割
    gkf = GroupKFold(n_splits=n_splits)

    val_indices = []

    for i, (_, val_idx) in enumerate(gkf.split(file_paths, groups=groups)):
        if i < n_val_splits:
            val_indices.extend(val_idx)
        else:
            break

    train_indices = set(range(len(file_paths))) - set(val_indices)

    train_files = np.array(file_paths)[list(train_indices)]
    val_files = np.array(file_paths)[list(val_indices)]

    return train_files, val_files

def copy_files(file_paths, dest_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    for file_path in file_paths:
        shutil.copy(file_path, dest_dir)

labels_dir = r"C:\Users\CorneAI\Desktop\Mobius_for_YOLOv8\labels"
train_files, val_files = group_k_fold_split(labels_dir)

# トレーニングセットとバリデーションセットのファイルをコピー
train_dir = os.path.join(labels_dir, 'train')
val_dir = os.path.join(labels_dir, 'val')

copy_files(train_files, train_dir)
copy_files(val_files, val_dir)

print("Files copied to Train directory:", len(os.listdir(train_dir)))
print("Files copied to Val directory:", len(os.listdir(val_dir)))

Files copied to Train directory: 2448
Files copied to Val directory: 1111


In [41]:
#一度消してやり直すとき
if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

if os.path.exists(val_dir):
    shutil.rmtree(val_dir)